- #### Transformaciones
    - ##### join
    - ##### UDF

In [ ]:
import org.apache.spark.sql.{SparkSession, DataFrame}

val spark = SparkSession.builder
        .appName("sesion_1")
        .master("local[*]")
        .getOrCreate()
val sc = spark.sparkContext

In [ ]:
sc.uiWebUrl

In [ ]:
def readCsv(path: String): DataFrame = {
    spark.read
        .option("header","true")
        .option("delimiter",",")
        .option("inferSchema","false")
        .csv(path)
}

val BasePath = "../../resources/data/csv/"
val clientsDf = readCsv(BasePath + "clients.csv")
val contractsDf = readCsv(BasePath + "contracts.csv")
val productsDf = readCsv(BasePath + "products.csv")

clientsDf.show()
contractsDf.show()
productsDf.show()

In [ ]:
// Joins

// inner -> Mantiene información de ambas tablas (columnas) para los registros (filas) coincidentes
// outer -> Mantiene información de ambas tablas (columnas y filas) para los registros coincidentes y no-coincidentes
// left -> Mantiene columnas de ambas tablas y filas únicamente de la tabla izquierda, elimina filas no coincidentes de la tabla derecha
// right -> Mantiene columnas de ambas tablas y filas únicamente de la tabla derecha, elimina filas no coincidentes de la tabla izquierda
// left_semi -> Mantiene filas y columnas únicamente de la tabla izquierda para los registros coincidentes
// left_anti -> Mantiene filas y columnas únicamente de la tabla izquierda para los registros no-coincidentes

// cross

In [ ]:
import org.apache.spark.sql.{functions => f}

val clientstmpdf = clientsDf.filter((f.col("edad") >= 40) && (f.col("edad") <= 50))
val contractsTmpDf = contractsDf.filter(f.col("activo") === "false")
    .withColumnRenamed("cod_titular", "cod_client")

clientstmpdf.show()
contractsTmpDf.show()

// clientstmpdf.crossJoin(contractsTmpDf).show() // WARNING

val JoinType = "full"   // inner, outer, left, right, left_semi, left_anti

val joinDf = clientstmpdf.join(contractsTmpDf, Seq("cod_client"), JoinType)
joinDf.show()

In [ ]:
// UDF - User Defined Function - WARNING
import org.apache.spark.sql.{types => t}
import scala.util.Try

val upperCaseFunction: String => Option[String] = value => Try(value.toUpperCase()).toOption

val len_concat: (String, String) => Int = (item_1, item_2) => {
    val EmptyString = ""
    val str1 = if (Option(item_1).isEmpty) EmptyString else item_1
    val str2 = if (Option(item_2).isEmpty) EmptyString else item_2
    
    str1.concat(str2).size
}

val upperUdf = f.udf(upperCaseFunction)

val lenConcatUdf = f.udf(len_concat)

joinDf.select(
    joinDf.columns.map(f.col) :+
    upperUdf(f.col("nombre")).alias("nombre_mayus") :+
    lenConcatUdf(f.col("nombre"), f.col("provincia")).alias("len_concat") :_*
).show()

In [ ]:
import org.apache.spark.sql.Row

val data = Seq(Row(1,null),
               Row(2,Double.NaN),
               Row(3,3.2),
               Row(4,Double.NaN))

val schema = t.StructType(Seq(
    t.StructField("id", t.IntegerType),
    t.StructField("number", t.DoubleType)))

val tmpDf = spark.createDataFrame(sc.parallelize(data), schema)

In [ ]:
tmpDf.show()
// Scala no puede revisar si un AnyVal es null, lo único que podemos hacer es procesar el NULL desde el api de SparkSQL

val isNullOrNan: Double => Boolean = value => if (value.isNaN) true else false
    
val isNullOrNanUdf = f.udf(isNullOrNan)

tmpDf.select(
    tmpDf.columns.map(f.col) :+
    isNullOrNanUdf(f.col("number")).alias("null_nan") :_*
).show()